# Approximation: Sampling


## Reservoir Sampling: Algorithm R

In [ ]:
# ported from https://en.wikipedia.org/wiki/Reservoir_sampling
from random import randrange

def reservoir_sample(data, n, k):
  # fill the reservoir array
  r = []
  for i in range(k):
    r.append(data[i])

  # replace elements with gradually decreasing probability
  for i in range(k, n-1):
    # randrange(a) generates a uniform integer in [0, a)
    j = randrange(i+1)
    if j < k:
        r[j] = data[i]
            
  return(r)
     
data = list(range(1000))
n = len(data)
k = 5
r = reservoir_sample(data, n, k)
r

Likelihood of each value 1 to 1000 to be in the database:

In [ ]:
import numpy as np
import pandas as pd

K = 5
N_SAMPLES = 10000
samples = []

N = 1000
data = list(range(N))
for j in range(N_SAMPLES):
    samples += reservoir_sample(data, N, k)

#unique, counts = np.unique(samples, return_counts=True)
ax = pd.Series(samples).plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
ax.set_title(f"Distribution of frequencies of all {N} values")

## Optimization to Reservoir: AlgorithmL
- Calling the random number generator for every row can be slow.
- Idea: after each row we choose, could we predict how many rows we'll skip?
- Let's plot the gaps between chosen values empirically! (The *sampling gap distribution*).

In [ ]:
import numpy as np
import pandas as pd

def plot_gaps(r):
    r.sort()
    gaps = pd.Series(np.diff(r))
    gaps.plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
data = list(range(100000))
n = len(data)
k = 1000
r = reservoir_sample(data, n, k)
plot_gaps(r)

- Turns out this is approximately a geometric distribution with a closed form!
  - Won't prove here
- So we can pick random gaps from geometric distribution
  - "skip over" the to-be-discarded inputs in between
  - only call RNG as many times as there are gaps!
  - i.e. about as many times as the size of the sample!

In [ ]:
# This is called Algorithm L
# ported from https://en.wikipedia.org/wiki/Reservoir_sampling
from random import random, randrange
from math import exp, log, floor

def reservoir_sample_L(data, n, k):
  # fill the reservoir array
  r = []
  for i in range(k):
    r.append(data[i])
    
  # random.random() generates a uniform [0,1) random number
  w = exp(log(random())/k)

  while i < n:
      i = i + floor(log(random())/log(1-w)) + 1
      if i < n:
          # replace a random item of the reservoir with item i
          r[randrange(k)] = data[i]  # random index between 0 and k-1, inclusive
          w = w * exp(log(random())/k)
            
  return(r)
     
data = list(range(1000))
n = len(data)
k = 5
r = reservoir_sample_L(data, n, k)
r

In [ ]:
data = list(range(100000))
n = len(data)
k = 1000
r = reservoir_sample_L(data, n, k)
plot_gaps(r)

## Reservoir Sampling as a Table-Valued UDF in PostgreSQL
We can implement a reservoir as a table function that returns (rownumber, pos) pairs and join with that to sample.

In [ ]:
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball

In [ ]:
%sql CREATE EXTENSION IF NOT EXISTS plpython3u; -- import extension

In [ ]:
%%sql
-- create the reservoir_swaps UDF --
DROP TYPE IF EXISTS reservoir_pair CASCADE;
CREATE TYPE reservoir_pair AS (rownum integer, pos integer);
CREATE OR REPLACE FUNCTION reservoir_swaps(k integer, n integer) RETURNS setof reservoir_pair
    AS $$
  # optimized reservoir sampling algorithm, Algorithm L
  from random import random, randrange
  from math import exp, log, floor
  # fill the reservoir array
  r = []

  for i in range(k):
    yield((i,i))
    
  # random.random() generates a uniform [0,1) random number
  w = exp(log(random())/k)

  while i < n:
      i = i + floor(log(random())/log(1-w)) + 1
      if i < n:
          # replace a random item of the reservoir with item i
          w = w * exp(log(random())/k)
          yield(i, randrange(k))  # rand om index between 0 and k-1, inclusive
            
  return(r)
    $$
    LANGUAGE 'plpython3u'
    VOLATILE
    RETURNS NULL ON NULL INPUT;


CREATE OR REPLACE FUNCTION reservoir_rows(k integer, n integer) RETURNS setof integer
  AS $$
     SELECT MAX(rownum) AS rownum
     FROM reservoir_swaps(k, n)
     GROUP by pos $$ 
LANGUAGE 'sql'
VOLATILE;

In [ ]:
%%sql
SELECT reservoir_rows(10, count(*)::integer) FROM batting;

In [ ]:
%%sql
WITH rrows AS (SELECT reservoir_rows(10, count(*)::integer) AS rows 
                 FROM batting),
     rbatting AS (SELECT row_number() over(), * 
                    FROM batting)
SELECT *
  FROM rbatting, rrows 
 WHERE row_number = rows;

In [ ]:
%%sql
WITH rrows AS (SELECT reservoir_rows(15, count(*)::integer) AS rows 
                 FROM batting),
     rbatting AS (SELECT row_number() over(), * 
                    FROM batting)
SELECT COUNT(*)
  FROM rbatting, rrows 
 WHERE row_number = rows;

# Stratified Sampling

In [ ]:
# See larger display
%config SqlMagic.displaylimit = 80

In [ ]:
%%sql
-- Stratified Sampling with Reservoirs
WITH grprows AS (SELECT teamid, reservoir_rows(10, COUNT(*)::integer) AS rows 
                   FROM batting 
                  GROUP BY teamid),
     rbatting AS (SELECT row_number() over(partition by teamid), * 
                    FROM batting)
SELECT *
  FROM rbatting b, grprows g
 WHERE row_number = rows
   AND b.teamid = g.teamid
 ORDER BY b.teamid;